In [6]:
import requests 
import re
from bs4 import BeautifulSoup 
import pandas as pd
import pymysql
import pymongo
from config import mongoConn, remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd

In [17]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(mongoConn) 
db = client.shows_db
collection = db.items 
title_filter = { 
    "title" : {"$regex": f'Pulp Fiction', "$options" :'i' } #this is like a sql LIKE statement
}
dict_title = collection.find(title_filter, { '_id': 0 } )
df = pd.DataFrame(dict_title)
df = df.drop_duplicates(subset='title', keep="first")
df

[]

In [ ]:
conn = pymysql.connect(host=f'{remote_db_endpoint}',  user=remote_db_user,  password=remote_db_pwd,  db=remote_db_name) 
mycursor = conn.cursor() 
mycursor.execute('SELECT * FROM streamingservices')
sdf = pd.DataFrame(mycursor) 
mycursor.close()
conn.close()
sdf.rename(columns={0:'Service_ID', 1:'Service_Name', 2:'Service_Type', 3:'Service_Img', 4:'Service_Url'}, inplace=True )
sdf.drop(columns=[5,6], inplace=True )
sdf

In [ ]:
dic_titles = df.to_dict(orient='records')
for title in dic_titles:
    title['services_meta'] = []
    for service in title['services']:
        try:
            title['services_meta'].append(sdf.loc[sdf['Service_Name'] == service].to_dict(orient='records')[0])
        except:
            print(f'{service} meta not found')
df = pd.DataFrame(dic_titles)